In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


## Webcam prediction

In [14]:
#live video

classifier = cv2.CascadeClassifier(f'haarcascade_frontalface_default.xml')

emotion_model = load_model('bestweight.h5')


emotion_dim = (48,48)

emotion_bw = False 

emotion_dict = {0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'sad', 4: 'disgust', 5: 'fear', 6: 'surprise'}

vid_frames = []
cap = cv2.VideoCapture(0)
cap_video = False

if cap_video == True:
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('EmotionDetector.mp4', fourcc,10, (int(width), int(height)))
while True: 
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1,1)
    clone = frame.copy()
    bboxes = classifier.detectMultiScale(clone)
    for i in bboxes: 
        x, y, width, height = i[0], i[1], i[2], i[3]
        x2, y2 = x+ width, y+height
        emotion_roi = clone[y:y2, x:x2]
        emotion_roi= cv2.cvtColor(emotion_roi, cv2.COLOR_BGR2GRAY)
        emotion_roi = cv2.resize(emotion_roi, emotion_dim, interpolation = cv2.INTER_CUBIC)
        emotion_roi = emotion_roi/255
        emotion_roi = emotion_roi.reshape(1, emotion_roi.shape[0], emotion_roi.shape[1], 1)
        #emotion predictions
        emotion_predict = emotion_model.predict(emotion_roi)[0]
        emotion_idx = np.argmax(emotion_predict)
        emotion_cat = emotion_dict[emotion_idx]
        emotion_conf = f'{round(np.max(emotion_predict)*100)}%'
        cv2.putText(clone, f'{emotion_cat}: {emotion_conf}', (x, y+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 2)
        cv2.rectangle(clone, (x,y), (x2,y2), (0,225,0), 1)
        
      
        
        
    cv2.imshow('LIVE', clone)
    vid_frames.append(clone)
    if cap_video == True:
        out.write(clone)
    if cv2.waitKey(1) & 0xFF ==ord('q'): 
        break 
cap.release()

cv2.destroyAllWindows()

if cap_video == True:
    out.release()

## Mobile Camera prediction provide IP in url

In [13]:
#live video

classifier = cv2.CascadeClassifier(f'haarcascade_frontalface_default.xml')

emotion_model = load_model('bestweight.h5')


emotion_dim = (48,48)

emotion_bw = False 

emotion_dict = {0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'sad', 4: 'disgust', 5: 'fear', 6: 'surprise'}

vid_frames = []
#url = 'http://192.168.43.105:8080/video'
#cap = cv2.VideoCapture(url)
cap = cv2.VideoCapture(0)
cap_video = False

if cap_video == True:
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('Tests/FaceDetector.mp4', fourcc,10, (int(width), int(height)))
while True: 
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1,1)
    clone = frame.copy()
    bboxes = classifier.detectMultiScale(clone)
    for i in bboxes: 
        x, y, width, height = i[0], i[1], i[2], i[3]
        x2, y2 = x+ width, y+height
        emotion_roi = clone[y:y2, x:x2]
        emotion_roi= cv2.cvtColor(emotion_roi, cv2.COLOR_BGR2GRAY)
        emotion_roi = cv2.resize(emotion_roi, emotion_dim, interpolation = cv2.INTER_CUBIC)
        emotion_roi = emotion_roi/255
        emotion_roi = emotion_roi.reshape(1, emotion_roi.shape[0], emotion_roi.shape[1], 1)
        #emotion predictions
        emotion_predict = emotion_model.predict(emotion_roi)[0]
        emotion_cat1 = emotion_dict[0]
        emotion_cat2 = emotion_dict[1]
        emotion_cat3 = emotion_dict[2]
        emotion_conf1 = f'{round(emotion_predict[0]*100)}%'
        emotion_conf2 = f'{round(emotion_predict[1]*100)}%'
        emotion_conf3 = f'{round(emotion_predict[2]*100)}%'
        cv2.putText(clone, f'{emotion_cat3}: {emotion_conf3}', (x+5, y+50), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 2)
        cv2.putText(clone, f'{emotion_cat1}: {emotion_conf1}', (x+5, y+30), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 2)
        cv2.putText(clone, f'{emotion_cat2}: {emotion_conf2}', (x+5, y+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 2)
        cv2.rectangle(clone, (x,y), (x2,y2), (0,225,0), 2)
        
      
        
        
    cv2.imshow('LIVE', clone)
    vid_frames.append(clone)
    if cap_video == True:
        out.write(clone)
    if cv2.waitKey(1) & 0xFF ==ord('q'): 
        break 
cap.release()

cv2.destroyAllWindows()

if cap_video == True:
    out.release()

In [10]:
for idx, i in enumerate(vid_frames): 
    cv2.imwrite(f'Tests/VidFrames/{idx}.png', i)

In [11]:
img_loc = [22,52,74, 103]

images = []
for i in img_loc:
    img = cv2.imread(f'Tests/VidFrames/{i}.png')
    images.append(img)
    

stack_img = np.hstack(images)
cv2.imwrite('Images/DemoStack.png', stack_img)

TypeError: Expected Ptr<cv::UMat> for argument 'img'